# Stochastic Floating-Point arithmetic

It is relatively well known in the scientific computing community that the use of finite-precision floating-point (FP) computations (as opposed to computations with real, infinite-precision numbers) can be the source of quality losses in the computed results.

Stochastic Arithmetic is one of many ways which can be used to diagnose FP-related problems. In standard IEEE-754-compliant Floating-Point Arithmetic, the result of each floating-point operation is rounded to the nearest representable floating-point value. When some floating-point operations produce results which are not representable as floating-point values (as is the case, for example of 1/3 in base 10, or 1/10 in binary representations), some information is lost during this rounding operation. Stochastic Arithmetic models this loss of accuracy using random variables.

More precisely, this notebook explores the CESTAC arithmetic [Jean Vignes and Michel La Porte. Error Analysis in Computing, 1974] in which the result of each FP operation $x\circ y$ is replaced with
$$
\bigcirc_{\text{rnd}}(x,y) = \left|\begin{array}{ll}
  \bigtriangledown(x\circ y) & \text{with probability $\displaystyle\frac12$}, \\[1em]
  \bigtriangleup(x\circ y) & \text{with probability $\displaystyle\frac12$},
\end{array}\right.
$$
where for any real value $a\in\mathbb{R}$, $\bigtriangleup(a)$ and $\bigtriangledown(a)$ respectively denote its upward- and downward-rounded floating-point values.

In this notebook, two ways are explored to implement CESTAC in Julia:
1. defining new types, implementing the same interface as floating-points, so that they can be used in the same way in algorithms,
2. defining a macro to rewrite existing code, already specialized for Floating-Point values, so that it now uses randomly rounded FP operations.

In a first stage, however, a few arithmetic-related building blocks should be defined.

In [1]:
using Formatting

# Basic Blocks for (stochastic) FP-arithmetic

The implementation proposed here follows the same overall design as the Verrou tool ([http://github.com/edf-hpc/verrou]()). The main ideas are outlined below.

An Error-Free Transformation (EFT) for a given operation $\circ \in \{+, -, \times\}$ is a transformation allowing to express
$$
(x, \delta) = \bigcirc_{\text{eft}}(a, b)
$$
such that
$$
\begin{align*}
&x + \delta = a\circ b,\\
&x = \lozenge(a\circ b),
\end{align*}
$$
where $\lozenge(x)$ denotes rounding $x\in\mathbb{R}$ to the nearest representable FP value, and all equalities written above are exactly valid (*i.e.* in the sense of real, infinite-precision computations).

Directed roundings for an operation $a\circ b$ can be computed using EFTs in the following way: let $(x, \delta) = \bigcirc_{\text{eft}}(a, b)$. One of three situations can happen:

- if $\delta = 0$, then no rounding occurred. In this case, $x = \lozenge(a\circ b) = a\circ b$.

- if $\delta > 0$, then a rounding occurred, which was directed downwards. In this case, $x = \lozenge(a\circ b) = \bigtriangledown(a\circ b)$ and $\bigtriangleup(a\circ b) = \text{successor}(x)$.

- conversely, if $\delta > 0$, then $x = \lozenge(a\circ b) = \bigtriangleup(a\circ b)$ and $\bigtriangleup(a\circ b) = \text{predecessor}(x)$.

This strategy is implemented in this part.


## Error-Free Transformations

### Generic algorithms

These algorithms shoud work for all FP representations.

In [2]:
# EFT for the sum of two numbers
# TwoSum algorithm by Knuth
function twoSum(a, b)
    x = a + b
    z = x - a
    y = (a - (x-z)) + (b-z)
    return (x,y)
end

twoSum(0.1, 0.3)

(0.4, -2.7755575615628914e-17)

In [3]:
# Error-free splitting of a FP number, by Dekker
# a = x+y, where x and y do not overlap
function split(a)
    # New methods should be added to compute the correct
    # constant for each FP representation
    #
    # factor = 1+2^s, where s is the mantissa length
    factor(x::Float64) = 134217729
    factor(x::Float32) = 4097
    
    c = factor(a) * a
    x = c - (c - a)
    y = a - x
    return (x, y)
end

split (generic function with 1 method)

In [4]:
# Simple test for the error-free split of a Float64 
let bin_repr, a=0.33
    bin_repr(x) = format("{:b}", Int(significand(x)*2^(53)))
    
    println(bin_repr(a))
    (x, y) = split(a)
    println(bin_repr(x), " ", bin_repr(y))
end

101010001111010111000010100011110101110000101000111110
101010001111010111000010100000000000000000000000000000 111101011100001010001111100000000000000000000000000000


In [5]:
# EFT for the product of two FP numbers
# TwoProd algorithm by Veltkamp
function twoProd(a, b)
    x = a * b
    (a1, a2) = split(a)
    (b1, b2) = split(b)
    y = a2*b2 - (((x - a1*b1) - a2*b1) - a1*b2)
    return (x,y)
end

twoProd(0.1, 5.)

(0.5, 2.7755575615628914e-17)

In [6]:
# Approximate transformation for the division
# The transformation in not exact (the error is not representable)
# but at least the sign of y is guaranteed to be correct
function twoDiv(a, b)
    x = a / b
    (p1, p2) = twoProd(x, b)
    y = (a-p1-p2)/b
    return (x, y)
end

twoDiv(1., 3.)

(0.3333333333333333, 1.850371707708594e-17)

### Specific algorithms for Float32

Since double precision (Float64) is available to run higher-precision computations, it should be used to compute more efficiently transformations of Float32 operations.

Since all implementations of such operations will have very similar structure, a macro is defined to help generate them.

In [7]:
macro float32_transformation(operator, name)
    quote
        function $(esc(name))(a::Float32, b::Float32)
            x = $operator(a, b)
            r = $operator(Float64(a), Float64(b)) - Float64(x)
            y = Float32(r)
            return (x,y)
        end
    end
end

@float32_transformation (macro with 1 method)

In [8]:
@float32_transformation(+, twoSum)

twoSum(0.1f0, 0.2f0)

(0.3f0, -7.450581f-9)

In [9]:
@float32_transformation(*, twoProd)

twoProd(0.1f0, 5.f0)

(0.5f0, 7.450581f-9)

In [10]:
@float32_transformation(/, twoDiv)

twoDiv(1.f0, 3.f0)

(0.33333334f0, -9.934108f-9)

## Rounding modes emulation

EFTs defined above are now used to emulate directed rounding, and then define a randomly rounded operation.

In [11]:
# Directed rounding
function round_dir(op, a, b, mode)
    ops = Dict(:+ => twoSum,
               :* => twoProd,
               :/ => twoDiv)
    mode_pred   = Dict(:up   => (err -> (err>0)),
                       :down => (err -> (err<0)))
    mode_change = Dict(:up   => nextfloat,
                       :down => prevfloat)
    (res,err) = ops[op](a,b)
    if (err != 0 && mode_pred[mode](err))
        res = mode_change[mode](res)
    end
    
    return res
end

round_dir (generic function with 1 method)

In [12]:
# Random rounding
round_rnd = let rng = MersenneTwister()
    function(op, a, b)
        return round_dir(op, a, b, rand(rng, [:up, :down]))    
    end
end

(::#7) (generic function with 1 method)

We can check that randomly rounded operations are non-deterministic. Re-evaluating this cell multiple times yields different results.

In [13]:
round_rnd(:+, 0.1, 0.2)

0.30000000000000004

However, only operations producing round-off errors are perturbed; when a result is representable, it is never necessary to round it.

In [14]:
round_rnd(:+, 0.125, 0.5)

0.625

## Accuracy estimation

Using such a stochastic arithmetic transforms any computation result into a realization of a random variable, whose variance allows estimating the results quality / stability. We follow here the estimator introduced in [Douglas Stott Parker. Monte Carlo Arithmetic: exploiting randomness in floating-point arithmetic. 1997]:
$$
\hat{s} = -\log_{10}\frac{\hat\sigma}{\hat\mu},
$$
where $\hat\mu$ and $\hat\sigma^2$ respectively denote the sample mean and variance. Of course, this metric is only useful if the computed result is non-zero.

The following macro uses 10 samples to estimate the number of reliable digits of a (scalar) computation. It returns a tuple containing:
- the averag result, as a string showing only reliable digits,
- the estimated number of reliable (decimal) digits.

In [15]:
# Estimated accuracy
macro reliable_digits(expr)
    return quote
        x = [($expr) for i = 1:10]
        mu = mean(x)
        sigma = std(x)
        s = -log10(sigma/abs(mu))
        
        if (s<1)
            return "@.0", s
        end
        f = @sprintf "{:.%de}" (Int(floor(min(s, 16)))-1)
        format(f, mu), s
    end
end

@reliable_digits (macro with 1 method)

An exact computation has an "infinite" number of reliable digits

In [16]:
@reliable_digits round_rnd(:+, 0.125, 0.5)

("6.250000000000000e-01", Inf)

An operation producing a round-off error is expected to have lost only its last significant digit.

In [17]:
@reliable_digits round_rnd(:+, 0.1, 0.2)

("3.00000000000000e-01", 15.908832279630328)

In [18]:
@reliable_digits round_rnd(:+, 0.1f0, 0.2f0)

("3.000000e-01", 7.0909166f0)

# Stochastic arithmetic using types

A first way to implement Stochastic Arithmetic in Julia consists in defining new types, which can then be transparently used in generic functions.

## SFloat64: stochastic arithmetic for double precision FP numbers

In [19]:
type SFloat64
    value :: Float64
end

import Base: +, -, *, /, zero, one, inv, convert, promote_rule, transpose, dot
+(a::SFloat64, b::SFloat64) = SFloat64(round_rnd(:+, a.value, b.value))
*(a::SFloat64, b::SFloat64) = SFloat64(round_rnd(:*, a.value, b.value))
-(a::SFloat64, b::SFloat64) = SFloat64(round_rnd(:+, a.value, -b.value))
/(a::SFloat64, b::SFloat64) = SFloat64(round_rnd(:/, a.value, b.value))

zero(x :: SFloat64) = SFloat64(0.)
zero(::Type{SFloat64}) = SFloat64(0.)
one(::Type{SFloat64}) = SFloat64(1.)

inv(x::SFloat64) = SFloat64(1.) / x
transpose(x::SFloat64) = x
dot(x::SFloat64, y::SFloat64) = x*y

convert(::Type{SFloat64}, x::Float64) = SFloat64(x)
convert(::Type{Float64}, x::SFloat64) = x.value
promote_rule(::Type{SFloat64}, ::Type{<:Number}) = SFloat64

promote_rule (generic function with 124 methods)

Note how this type can be used in any generic algorithm, for example Linear Algebra (provided that specific methods are declared for all relevant functions)

In [20]:
let
    A = [1. 2. 3.
         4. 5. 6.
         7. 8. 10.]
    B = [1. ; 2. ; 3]

    X_ieee = A \ B
    @show X_ieee
    
    X_cestac = SFloat64.(A) \ SFloat64.(B)
    @show X_cestac
    
    Void()
end

X_ieee = [-0.333333, 0.666667, -0.0]
X_cestac = SFloat64[SFloat64(-0.333333), SFloat64(0.666667), SFloat64(0.0)]


### SFloat32: stochastic arithmetic for single-precision FP numbers

In [21]:
type SFloat32
    value :: Float32
end

import Base: +, -, *, /, zero, one, inv, convert, promote_rule, transpose, dot
+(a::SFloat32, b::SFloat32) = SFloat32(round_rnd(:+, a.value, b.value))
*(a::SFloat32, b::SFloat32) = SFloat32(round_rnd(:*, a.value, b.value))
-(a::SFloat32, b::SFloat32) = SFloat32(round_rnd(:+, a.value, -b.value))
/(a::SFloat32, b::SFloat32) = SFloat32(round_rnd(:/, a.value, b.value))

zero(x :: SFloat32) = SFloat32(0.)
zero(::Type{SFloat32}) = SFloat32(0.)
one(::Type{SFloat32}) = SFloat32(1.)

inv(x::SFloat32) = SFloat32(1.) / x
transpose(x::SFloat32) = x
dot(x::SFloat32, y::SFloat32) = x*y

convert(::Type{SFloat32}, x::Float32) = SFloat32(x)
convert(::Type{Float32}, x::SFloat32) = x.value
promote_rule(::Type{SFloat32}, ::Type{<:Number}) = SFloat32

promote_rule (generic function with 125 methods)

In [22]:
let
    A = [1. 2. 3.
         4. 5. 6.
         7. 8. 10.]
    B = [1. ; 2. ; 3]

    X_ieee = Float32.(A) \ Float32.(B)
    @show X_ieee
    
    X_cestac = SFloat32.(A) \ SFloat32.(B)
    @show X_cestac
    
    Void()
end

X_ieee = Float32[-0.333333, 0.666667, -0.0]
X_cestac = SFloat32[SFloat32(-0.333333), SFloat32(0.666666), SFloat32(4.76836f-7)]


## Test: dot product computation

As a test, we try computing dot products. Two data sets are used here, each defining two 100-element vectors with various condition numbers.

Data used in this test have been obtained using the algorithm proposed in [Ogita, Rump and Oishi. Accurate Sum and Dot Product. 2005].



### Well-conditioned case

The condition number of this dot product is approximately 1.8e3, so that it is to be expected that 3 significant (decimal) digits will be unreliably computed. The result should therefore contain 13 reliable significant digits in double precision, or 4 in single precision.

In [23]:
x1 = [-6.340663515282668472e-01, -1.032478304663339008e+00, -9.398547076618038787e+00, 8.775666080119043144e+00,
      8.154021046137667206e-01, -2.872197835897075890e+00, -4.130201373187722957e+00, -3.800523733887637978e-01,
      3.767316625622183501e+00, -7.693130974005489620e+00, 3.789995392417770503e+00, 2.332943819408263633e-02,
      1.117290458364406547e+00, -1.029424506285847585e+01, -1.972399240364107298e+00, 2.409324890261351548e+00,
      1.514218404563302478e+01, -9.803395024842704863e-01, 3.624014860482689393e+00, 3.454279166512129517e+00,
      2.150407406181448877e+00, 8.380973701960218181e-01, -4.718497388831256245e+00, 1.092357091710888817e+01,
      6.201621778907238092e-01, -2.431850975835210704e+00, -9.587862761576261272e-01, -4.744452383420612485e+00,
      -1.928632137138599045e+00, -1.514994713586201414e+00, -2.183297984924969626e+00, 1.424576004184546996e+01,
      9.692913616904123231e+00, -1.547104695550167275e+00, 4.204554274223821331e+00, 9.347482098474469980e-01,
      7.399023959120087923e-02, -4.191315817777256925e+00, 8.475563783170365184e+00, -1.721899852461975300e+00,
      -7.486679611973100279e+00, -5.787527477378286850e-02, 5.560569808434617123e+00, 9.797972685704890716e-02,
      -1.002061593959254537e+00, -3.012224958934606622e+00, -8.084325856937959465e-01,-1.905132695346669347e-01,
      5.188198608642882181e-01, -7.601644981994752914e-01, -2.451000778662956403e+00, -9.693293689094003573e-01,
      2.882117571206391560e+00, 2.269843085532857518e+00, 2.958122951288597147e+00, 2.630063037097047030e+00,
      -6.039968388496026463e+00, -4.780642850877452332e+00, 2.900723065170244808e-01, -1.902925253860966448e+00,
      -1.317310182590558298e+01, 3.598067326176326297e-01, 9.089778308264061524e-01, -3.252711812214260867e+00,
      -2.310740125115115173e+00, -1.617256866537321480e+00, 2.935716535944118633e+00, -3.234285498797106584e+00,
      3.237522392256278359e+00, 4.234360820593143337e+00, -8.323039886529393083e-02, 3.392892532427555174e+00,
      4.675163615855799232e-02, -2.586710584045738504e+00, 7.263314423520003116e-01, -1.805428377059409950e+00,
      5.540332712174778074e-01, -1.680124146046180877e+00, 3.384520587773187650e-01, -9.285586956857446950e+00,
      1.226698449940088764e+01, 1.955258288381735765e+00, -1.751307418060096488e+00, 2.822752021460029237e+00,
      3.284769252355420832e+00, -1.435237503511320289e+00, -1.640731710755447281e+01, -5.073426138128573903e+00,
      2.769683815669464200e-01, -2.364893385542928161e+00, -2.852226279751059579e+00, 9.442997895825611110e-02,
      1.613660165004466629e+00, -7.535249859542684225e-01, 1.875042460291437951e+00, -1.204955334660282817e+00,
      -1.312339352451679320e+00, 1.029598837959142621e+01, 1.181600593671352062e+00, -1.298538943700622506e+00]

y1 = [2.904271758197305431e+00, -7.586923460587383872e+00, -2.433948194217723793e-01, 7.211094113495739588e-01,
      3.682896270200847866e+00, -1.300501673307355999e+00, -1.156385074689293901e+01, 3.527526896240007925e+00,
      6.138604794505512219e-01, 3.789599979884540115e+00, 4.964840840608681027e+00, 1.003350838878128926e+00,
      4.318670472060466281e+00, 1.014498387743745011e+00, -2.234343190283604841e+00, -6.082736396317145466e-01,
      2.676279050222233336e-01, -2.474834547943665974e+00, -1.502899068454430775e+00, -1.853840253722779163e+00,
      -2.690201016281298152e+00, -2.762974865876188346e+00, -1.691254623016295255e+00, 7.037535285788961217e-03,
      8.664535245217100945e-01, -1.152357491478533280e+00, 1.028831455069202150e+00, -1.999749826926913443e-01,
      -4.083433509619152524e+00, 8.094079296833567305e-01, 1.548542269669347382e+00, 1.980881016229119940e+00,
      8.859099623710096072e-01, 1.071154721221595274e+01, -9.323551904078861141e-02, 1.071342262133416101e+00,
      -4.657941295025913031e-01, -1.417360280450415067e+00, 2.973792467793110017e+00, -5.333088668455037151e-01,
      3.506402422142353981e+00, -2.626404402475698419e+01, -5.821956212486361082e+00, -2.081525484012217042e+00,
      1.166141951259942111e+01, -2.097572645415919368e-01, -4.170975773332724401e+00, -1.088414007670470252e+00,
      1.437306693315153927e+00, 2.525907646776998305e+00, -3.752587514423713078e+00, 1.547393157729613611e+00,
      5.679913072629369508e-02, -4.458331660254577500e+00, -1.152536727954037943e+00, 6.959027574408890082e+00,
      -4.529530640152803933e+00, 2.850439979915360689e-01, 1.551842054260149303e-02, -2.499408640456699970e+00,
      8.025705275120998294e+00, -2.339259319371728374e+00, 1.690685407197963785e-01, 1.773766221659853759e+00,
      8.418125128249426270e-02, -1.240164986459572516e+00, 1.244333273760872993e+00, 2.229039874696919110e+00,
      1.578366388587486036e+01, 4.010353714690342741e+00, 4.066297377969993221e+01, -3.590142776920160483e-01,
      -5.317967102854119776e-01, -5.773767334043824473e-01, -1.684750638812332113e+00, 3.036554271013544026e-01,
      -1.915078321632401881e+00, 1.230841242088420717e+00, -1.360733255768588945e+00, -2.194897100045668736e+00,
      -3.147503241618139214e-01, -6.491785562479393867e+00, 8.732579404206023410e-01, -1.197064788553268588e+00,
      -1.051560432555771030e+00, 3.857770155380043642e-01, 1.780083978520605070e+00, 6.985067850240034293e-01,
      5.260718423653707454e+00, 2.393421981789398967e-01, 1.226572487102020936e+00, -4.390511466037616550e-01,
      4.857518183853492544e-01, 9.529898633767956539e+00, 1.640668058073694624e+01, 1.437731392011400100e+00,
      2.081762488795357413e+00, -1.677143583187016507e+00, 3.714458425965566635e-01, -1.788480931953309927e+00]

100-element Array{Float64,1}:
   2.90427 
  -7.58692 
  -0.243395
   0.721109
   3.6829  
  -1.3005  
 -11.5639  
   3.52753 
   0.61386 
   3.7896  
   4.96484 
   1.00335 
   4.31867 
   ⋮       
   5.26072 
   0.239342
   1.22657 
  -0.439051
   0.485752
   9.5299  
  16.4067  
   1.43773 
   2.08176 
  -1.67714 
   0.371446
  -1.78848 

In [24]:
# Standard floating-point evaluation (deterministic)
dot(x1, y1)

-0.8834885213601043

In [25]:
# Stochastic evaluation (non-deterministic)
dot(SFloat64.(x1), SFloat64.(y1))

SFloat64(-0.8834885213601695)

In [26]:
# Estimation of the result quality
@reliable_digits dot(SFloat64.(x1), SFloat64.(y1)).value

("-8.834885213601e-01", 13.301048090294726)

In [27]:
@reliable_digits dot(SFloat32.(x1), SFloat32.(y1)).value

("-8.835e-01", 4.416569f0)

### Ill-conditioned case

The condition number of this dot product is approximately 3e12, so that it is expected that 12 (decimal) digits are unreliably computed. Only 3 digits should remain reliable in the double-precision results; no digit is expected to be reliably computed in single precision.

In [28]:
x2 = [-4.703899089072115913e+01, -2.302208194432241726e+03, -1.564726108041984262e+05, 1.526300899208613373e+04,
      -2.132307857407913616e+02, 2.307518097958435537e+05, -5.192572714570803782e+01, 2.325599833211251966e+00,
      2.817494622670464651e+03, 1.708710020563955254e+02, 6.269522630918198303e+01, 3.687443650750188340e+01,
      1.038604765677104957e+01, -1.915975573744401117e+03, 1.168276776869518835e+04, 2.454491144842770009e+05,
      -5.592615927834867051e+01, 1.850307202352605032e+02, -3.838926631532003521e+05, 4.581024635636574203e+03,
      -4.418458866400893044e+02, -1.291545011255889767e+04, -2.048725390018565307e+01, 1.204863349219630436e+03,
      9.760401372457109259e-01, 1.410078033770042748e+04, -4.813837462145154859e+04, -1.437299278084431897e+01,
      -5.155141433885022707e+03, -2.255002085683215992e+05, 5.356346283817102085e+04, 3.667837584637414693e+04,
      -7.216672700119137880e+04, -1.259108454694217698e+03, 4.419830333598044092e+01, -3.585915213658949360e+05,
      3.816928321706653549e+01, 2.135141050205000557e+00, 1.174394551478064532e+05, -2.710374703222661275e+00,
      1.611291396027762385e+05, -1.357925813584457501e+03, -1.242968716437326115e+01, 7.973254168167831267e+03,
      -1.866706364323428602e+02, 2.042441529597737571e+01, 2.158858184523954282e+00, 2.467055660032614028e+01,
      5.375520883445242362e+04, -1.144212376192913938e+00, 5.994613017286798140e+02, 3.193496564759223055e+00,
      -4.148337081370820911e-01, -4.005110071830330076e-01, 1.993341984143252921e+04, -1.147244379776030421e+01,
      6.119929339861500672e-01, -6.092377012266528027e+02, 2.411059939457267774e+03, 9.559736934852315926e+01,
      -4.805764149222165997e+02, 4.009221133304038085e+04, 8.398867979989321775e-02, 5.743855858417433069e+01,
      -4.980311032420013362e+02, 1.509343053835127830e+04, 1.454679269769394365e-02, 2.854821043634199995e+01,
      -4.072883091111050291e-01, 5.108044955045782842e+01, -4.071539984133301004e+03, 8.751867981322608614e+01,
      -1.285130410507232591e+05, -3.354125317288757287e+03, 8.135505746482945000e-01, 4.723687613018444154e+01,
      1.189127895476845197e+03, -2.595180439442086026e+03, -3.753511234127828402e+02, 6.828869753373305151e+00,
      -2.424235269921588838e+02, -1.527378814146540753e+00, -2.753704293718250701e+05, 6.474506937411601548e+00,
      -1.077862672807065770e+06, -4.139018884967313352e+04, -1.418115653149918671e+03,-1.042964882907427171e+00,
      1.508774032010480548e+00, -1.001265318103480295e+00, 2.624649797909716753e+00, -1.368302828037557447e+04,
      -6.551342612757771349e+02, -1.855860091170598389e+04, 3.231487400789144715e+01, 9.365283668012376438e+03,
      1.078202387444531450e+04, 1.383253991429271628e+04, 1.406923835053669469e+00, -5.288835748811105963e-01]

y2 = [-1.252871778887357568e+00, 1.078962521009007787e+03, -3.174894745849149331e+04, -1.530373455916264093e-02,
      4.784478272815556821e+00, -1.192086818719253206e+05, 3.668476944821762942e+01, 1.952404524146996501e+00,
      4.765260911366479377e+03, -3.289741608770166970e-01, -4.226193799391714911e+00, -1.824747767003190901e+01,
      1.352880351874526710e+00, 2.528843202632653629e+03, -1.591117252770609957e+04, -7.948333831513273440e-01,
      1.763730594001834406e+00, -4.142264664747529190e-01, 1.066477328704419342e+05, 3.159332045246057532e+03,
      -1.553414963997087783e+00, 6.473297499671324395e-01, -5.969956700965419749e+00, -1.432466768506058770e+03,
      3.846365749072568452e+00, 2.790519287535665249e+00, 2.712376392123455826e+00, 3.495804741318556119e-01,
      -1.976446781200929399e+04, 1.520383644699746801e+05, 2.197669377729083135e+04, -1.069924380149832541e+04,
      -1.364616481795411164e+04, 2.396787353506200924e-01, 1.576007089234049907e+01, -1.640112041994733736e+06,
      5.423184829884636571e+00, -2.854528605476235548e+00, 1.147783795765330961e+00, -1.108619081995681377e+00,
      -1.290633352716010995e+05, 8.452162075538846020e-01, -1.339879936618119416e+02, -1.094006209459130559e+03,
      2.012798587947189688e+00, -9.383522798994409220e-01, -1.090080505163032232e+00, 2.617736898339717300e+01,
      1.163689330823294954e+00, -1.997489359474004189e+01, -1.497673101858773350e+02, 2.910347668348660832e+01,
      6.358777943232427621e-01, -3.223860439180939785e+00, -1.020470547974752958e-02, -2.573297990895500220e+01,
      1.219805387956967024e+00, 8.361151282404189260e+02, 3.744876783652914298e+01, -2.384432846910274861e+01,
      4.299066016316168692e+00, -3.581774420101832220e+04, 6.850586911579179095e+00, 1.991718463645072712e-01,
      -7.417107650666663403e+02, -7.336543124226103828e+03, -5.102552005287688264e-01, 4.026602472160422508e+01,
      3.045402731323658330e+01, 6.217979244611119327e+01, 4.537584779899785644e+03, -6.591459843464705592e+00,
      3.949078543721616708e+00, -1.752538562659680388e-01, 5.175050101609877107e-01, 9.134566870551368822e+01,
      2.127631922755461868e+00, -3.851289922958672785e-01, -5.430969234878427443e+01, 2.599543161451561613e+00,
      2.418636072495988856e+02, 2.920924458066446405e+00, -3.498182839331332798e+04, 1.412711102346127312e+01,
      4.450907201869644923e+05, -4.828728797199047129e+00, 4.874121684200859050e+01, 6.351064829043707505e+00,
      -1.052720771587801307e+00, 4.862959846339910541e+01, -5.724214959936722025e-01, 1.159505461894495326e+00,
      1.220304567286626707e+02, -8.614815388556389397e-01, -2.020054259416569076e-01, 7.288236435882137130e+02,
      -4.939913027080400365e-01, 2.953432423721218220e+04, -5.565878606086428704e+00, 6.137681992750880777e-01]

100-element Array{Float64,1}:
     -1.25287  
   1078.96     
 -31748.9      
     -0.0153037
      4.78448  
     -1.19209e5
     36.6848   
      1.9524   
   4765.26     
     -0.328974 
     -4.22619  
    -18.2475   
      1.35288  
      ⋮        
     -1.05272  
     48.6296   
     -0.572421 
      1.15951  
    122.03     
     -0.861482 
     -0.202005 
    728.824    
     -0.493991 
  29534.3      
     -5.56588  
      0.613768 

In [29]:
dot(x2, y2)

0.8098193831511069

In [30]:
dot(SFloat64.(x2), SFloat64.(y2))

SFloat64(0.8100831933088838)

In [31]:
@reliable_digits dot(SFloat64.(x2), SFloat64.(y2)).value

("8.10e-01", 3.542207289203493)

In [32]:
@reliable_digits dot(SFloat32.(x2), SFloat32.(y2)).value

("@.0", -0.27877644f0)

# Stochastic Arithmetic using code transformations

In case an algorithm has already been written with explicit Floating-Point types, it is not possible to simply change the data type and rely on Julia's multiple dispatch. In this case however, it is possible to rewrite the AST on-the-fly in order to create a new variant of the function, in which all FP operations are replaced by their stochastic equivalent.

More precisely, the following replacements are performed:
- `x + y` → `my_sum(x, y)`
- `x * y` → `my_prod(x, y`
- `x += y` → `x = my_sum(x, y)`
- `x *= y` → `x = my_prod(x, y)`

In [33]:
my_prod(x, y) = x * y
my_prod(x::Float64, y::Float64) = round_rnd(:*, x, y)
my_prod(x::Float32, y::Float32) = round_rnd(:*, x, y)

my_sum(x, y) = x + y
my_sum(x::Float64, y::Float64) = round_rnd(:+, x, y)
my_sum(x::Float32, y::Float32) = round_rnd(:+, x, y)

macro mca(defun)
    transform!(x) = Void()
    transform!(e::Expr) = begin
        for i in 1:length(e.args)
            if typeof(e.args[i]) == Expr && e.args[i].head == :(+=)
                ee = e.args[i]
                transform!.(ee.args)
                e.args[i] = :($(ee.args[1]) = $(ee.args[1]) + $(ee.args[2]))
            elseif typeof(e.args[i]) == Expr && e.args[i].head == :(*=)
                ee = e.args[i]
                transform!.(ee.args)
                e.args[i] = :($(ee.args[1]) = $(ee.args[1]) * $(ee.args[2]))
            elseif e.args[i] == :*
                e.args[i] = :my_prod
            elseif e.args[i] == :+
                e.args[i] = :my_sum
            end
            transform!(e.args[i])
        end
    end
        
    transform!(defun)
    println(defun)
    defun
end


@mca (macro with 1 method)

## Test

We test the previous macro on a (very naive) mixed-precision implementation of the dot product: the dot product of two single-precision vectors is computed using double-precision for the accumulation of partial sums. Note that, since some care has been taken to select specific Floating-Point precisions for different parts of the computation, estimating the results quality with the SFloat64 type would require adapting the function definition. 

In [34]:
@mca function mixed_dot(x::Array{Float32,1}, y::Array{Float32,1})
    res = 0.0 ::Float64
    for i in 1:length(x)
        res += x[i] * y[i]
    end
    return res
end

function mixed_dot(x::Array{Float32, 1}, y::Array{Float32, 1}) # In[34], line 2:
    res = 0.0::Float64 # In[34], line 3:
    for i = 1:length(x) # In[34], line 4:
        res = my_sum(res, my_prod(x[i], y[i]))
    end # In[34], line 6:
    return res
end


In [35]:
@reliable_digits mixed_dot(Float32.(x1), Float32.(y1))

("-8.8348e-01", 5.126652975794268)

As a last example, the macro is used to estimate the quality of a compensated dot product algorithm [Ogita, Rump and Oishi. Accurate Sum and Dot Product. 2005]. This algorithm is expected to compute results "equivalent to" a standard algorithm in twice the working precision.

In [36]:
@mca function comp_dot(x, y)
    res = zero(x[0])
    err = zero(x[0])
    for i = 1:length(x)
        (p,   err1) = twoProd(x[i], y[i])
        (res, err2) = twoSum(res, p)
        err += err1 + err2
    end
    return res + err
end

function comp_dot(x, y) # In[36], line 2:
    res = zero(x[0]) # In[36], line 3:
    err = zero(x[0]) # In[36], line 4:
    for i = 1:length(x) # In[36], line 5:
        (p, err1) = twoProd(x[i], y[i]) # In[36], line 6:
        (res, err2) = twoSum(res, p) # In[36], line 7:
        err = my_sum(err, my_sum(err1, err2))
    end # In[36], line 9:
    return my_sum(res, err)
end


In [37]:
@reliable_digits comp_dot(x2, y2)

("8.09888192127440e-01", 15.990651089900688)

In [38]:
@reliable_digits comp_dot(Float32.(x2), Float32.(y2))

("2.60827e+04", 6.9218807f0)